In [1]:
import sympy as sym
import numpy as np
import pandas as pd

**Objectif** - rejeter rapidement les densités ne disposant pas d'une masse à l'infini. Pour les densités paramétriques, obetnir des conditions suffisantes pour rejeter une expression.

In [2]:
def quick_asymptotic_test(rho_expr: sym.Expr, var: sym.Symbol, vals_dict: dict):
    """ Does a quick asymptotic check 
    """
    
    sub_rho_expr = rho_expr.subs(vals_dict)

    # Check asymptotic behavior using classic rules
    sub_rho_expr = sym.simplify(var**3 * sub_rho_expr)
    try:
        L = sym.limit(sub_rho_expr, var, sym.oo)
    except Exception:
        L = None

    if L is sym.oo:
        return "infinite"

    if L == 0:
        return "maybe-finite"

    if L is not None and L.is_real and L.is_finite and L != 0:
        return "infinite"

    return None

r = sym.symbols('r', positive=True, real=True)
alpha = sym.symbols('alpha', positive=True, real=True)

rho_expr = 1 / (1 + r**alpha)
quick_asymptotic_test(rho_expr, r, {alpha:4})

'maybe-finite'

On ne peut a priori pas rejeter la densité $\frac{1}{1 + r^4}$. On peut en fait la conserver, car $\sim C r^{-4}$ - on propose un test qui extrait cette puissance.

In [3]:
def get_infinite_mass_condition(rho_expr: sym.Expr, var: sym.Symbol, vals_dict: dict = None):
    """ Given parametric density, tries to determine
        values for which the mass is indefinite.
    """
    result = {
        'rho': None,
        'is_indefinite': None,
        'condition': None
    }

    # Substitute values if availible
    rho = rho_expr
    if vals_dict is not None:
        rho = rho.subs(vals_dict)
    result['rho'] = rho

    # Sub-step - No free parameters: a quick check is possible
    all_syms = rho.free_symbols
    param_syms = [s for s in all_syms if s!=var]

    if not param_syms:
        quick_test = quick_asymptotic_test(rho, var, {})
        if quick_test == 'infinite':
            result['is_indefinite'] = True
            return result

    # Main-step - compute logarithmic slope
    rho = sym.simplify(sym.Abs(rho))
    try:
        slope = sym.limit(sym.log(rho)/sym.log(var), var, sym.oo, dir='+')
    except Exception:
        return result
    
    if slope.is_infinite:
        result['is_indefinite'] = False
        return result
    
    if slope is sym.nan:
        return result
    
    p = -slope
    cond = sym.Le(p, sym.Integer(3))

    cond = sym.simplify(cond)
    if cond==True or cond==False:
        result['is_indefinite'] = cond
        return result

    result['condition'] = cond

    return result

r = sym.symbols('r', positive=True, real=True)
alpha = sym.symbols('alpha', positive=True, real=True)
beta = sym.symbols('beta', positive=True, real=True)
gamma = sym.symbols('gamma', positive=True, real=True)

rho_expr = 1/(r**alpha * (2+r**(1/beta))**(beta*(gamma-alpha)))
result = get_infinite_mass_condition(rho_expr, r)

print("Condition: ", result['condition'])
print("Is indefinite: ", result['is_indefinite'])

Condition:  gamma <= 3
Is indefinite:  None


Pour le modèle DiCintio, une condition suffisante pour rejeter la densité est $\gamma \leq 3$.

In [4]:
rho_expr = sym.exp(-2/alpha * (r**alpha - 1))
result = get_infinite_mass_condition(rho_expr, r)

print("Condition: ", result['condition'])
print("Is indefinite: ", result['is_indefinite'])

Condition:  None
Is indefinite:  False


Sans surprise, pour le modèle Einasto, l'intégrale est toujours définie pour $\alpha > 0$. A noter que si on ne suppose pas que $\alpha$ est positif, le calul de la limite échoue, car le résultat dépend du signe de $\alpha$.

In [5]:
rho_expr = 1 / (1 + r**alpha)
result = get_infinite_mass_condition(rho_expr, r, {alpha:3})

print("Condition: ", result['condition'])
print("Is indefinite: ", result['is_indefinite'])

Condition:  None
Is indefinite:  True


Si la fonction `get_infinite_mass_condition` renvoie deux `None`, alors cela signifie que le test rapide $\lim_{r\to \infty} r^3 \rho(r)$ n'a pas convergé vers une valeur non-nulle ou divergé, et que le calcul d'une puissance asymptotiquement équivalente a échoué. Dans ce cas, on ne peut pas clairement décidé de l'intégrabilité. 

In [6]:
from sympy.parsing.sympy_parser import parse_expr

profiles = {
    "NFW": ("rho0 / ((r / Rs) * (1 + r / Rs)**2)", ['rho0', 'Rs']),
    "superNFW": ("rho0 / ((r / Rs) * (1 + r / Rs)**Rational(5, 2))", ['rho0', 'Rs']),
    "pISO": ("rho0 / (1 + (r / Rs)**2)", ['rho0', 'Rs']),
    "pISO1": ("1 / (1 + (r/1)**2)", []),
    "Burkert": ("rho0 * Rs**3 / ((r + Rs)*(r**2 + Rs**2))", ['rho0', 'Rs']),
    "Lucky13": ("rho0 / (1 + (r/Rs))**3", ['rho0', 'Rs']),
    "Einasto": ("rho0 * exp(-2/a * ( (r/Rs)**a - 1 ) )", ['rho0', 'Rs', 'a']),
    "coreEinasto": ("rho0 * exp(-2/a * ((r/Rs + rc/Rs)**a - 1))", ['rho0', 'Rs', 'a', 'rc']),
    "DiCintio": ("rho0 / ( (r/Rs)**a * (1+(r/Rs)**(1/b))**(b*(g-a)))", ['rho0', 'Rs', 'a', 'b', 'g']),
    "gNFW": ("rho0 / ((r/Rs)**g * (1 + r/Rs)**(3-g))", ['rho0', 'Rs', 'g']),
    "Dekel-Zhao": ("rho0 / ((r/Rs)**a * (1 + (r/Rs)**(1/2))**(7-2*a))", ['rho0', 'Rs', 'a']),
    "Exponential": ("rho0 * exp(-r/Rs)", ['rho0', 'Rs']),
    "Exponential1": ("9.6 * exp(-r/1.4)", []),
    "Exponential2": ("rho0 * exp(-r/(Rs_1 + Rs_2))", ['rho0', 'Rs_1', 'Rs_2']),
}

r = sym.symbols('r', positive=True, real=True)
rho0 = sym.symbols('rho0', positive=True, real=True)
Rs, Rs_1, Rs_2 = sym.symbols('Rs Rs_1 Rs_2', positive=True, real=True)
rc = sym.symbols('rc', positive=True, real=True)
a, b, g = sym.symbols('a b g', positive=True, real=True)

local_dict = {
    'r': r, 'rho0': rho0, 'Rs': Rs, 'Rs_1': Rs_1,
    'Rs_2': Rs_2, 'rc': rc, 'a': a, 'b': b, 'g': g
}

global_vals_dict = {
    Rs: 1, Rs_1: 1, Rs_2: 1, rc: 1, rho0: 1
}

for rho_name in profiles:
    print(f"### {rho_name} ###")
    rho_expr = sym.simplify(parse_expr(profiles[rho_name][0], local_dict=local_dict))

    result = get_infinite_mass_condition(
        rho_expr=rho_expr,
        var=r,
        vals_dict=global_vals_dict
    )

    print("Condition: ", result['condition'])
    print("Is indefinite: ", result['is_indefinite'], '\n')


### NFW ###
Condition:  None
Is indefinite:  True 

### superNFW ###
Condition:  None
Is indefinite:  False 

### pISO ###
Condition:  None
Is indefinite:  True 

### pISO1 ###
Condition:  None
Is indefinite:  True 

### Burkert ###
Condition:  None
Is indefinite:  True 

### Lucky13 ###
Condition:  None
Is indefinite:  True 

### Einasto ###
Condition:  None
Is indefinite:  False 

### coreEinasto ###
Condition:  None
Is indefinite:  False 

### DiCintio ###
Condition:  g <= 3
Is indefinite:  None 

### gNFW ###
Condition:  None
Is indefinite:  True 

### Dekel-Zhao ###
Condition:  None
Is indefinite:  False 

### Exponential ###
Condition:  None
Is indefinite:  False 

### Exponential1 ###
Condition:  None
Is indefinite:  False 

### Exponential2 ###
Condition:  None
Is indefinite:  False 

